In [ ]:
using Images, Colors, Distributions

function muestra(radio=2.0, centro=0.0im, alto=1000, ancho=1000)
    dx=(2.0*radio)/(1.0*ancho) #el ancho que representa un pixel
    dy=(2.0im*radio)/(1.0*alto)#el alto que representa un pixel
    z0 = centro + radio*(1.0im - 1.0) #el primer valor de la muestra es la esquina superior izquierda
    return [z0 + (x*dx) - (y*dy) for y in 1:alto, x in 1:ancho]
end

# Ejemplo 1: **El método de Newton**

Veamos un ejemplo de dibujitos dinámicos con información *apriori*. 

Nos recordamos que el método de Newton es un algoritmo para aproximar raíces de funciones holomorfas. 
Sea $r$ una raíz de la función $f(r)=0$ y $z$ una aproximación de la raíz.
Considerando la expansión en serie de potencias de $f$ alrededor de $z$, $$ f(x) = f(z) + f'(z) (x - z) + O((x-z)^2), $$ 
y suponiendo que $z$ esta suficientemente cerca de $r$ como para que $O((z-r)^2)$ sea un error descartable, para $x=r$ tenemos $ 0 = f(r) = f(z)  + f'(z) (r - z) + \epsilon $ y podemos obtener una nueva estimación para $r$ como $$ z^* := r + \varepsilon = z - \frac{f(z)}{f'(z)}.$$
De esta forma, definimos el metodo de Newton para $f$ como la función $$N_f(z) = z - \frac{f(z)}{f'(z)}$$

## Algunos detalles curiosos del método de Newton

- La derivada del método de Newton es  $$N'_f (z) = \frac{f(z) f''(z)}{(f'(z))^2}.$$

- Los puntos fijos de $N_f$ son la soluciones de la ecuación $$ \frac{f(z)}{f'(z)} = 0, $$ que incluyen las raíces de $f$ y el punto $\infty$.

- Las raices de $f$ son atractores (superatractores si son raíces simples) y el punto $\infty$ es repulsor.

Para empezar vamos a ver el caso de polinomios complejos. 
Haremos el dibujito de las cuencas de atracción de las raíces del polinomio conociendo de antemano cuáles son las raíces.

Si aplicamos el método de Newton a polinomios, podemos expresar $N_{p}$ como $$ N_p(z) = z - \cfrac{1}{\sum_{i=1}^ {d}\frac{1}{z - \rho_i}} $$ donde $\rho_i$ es la $i$-ésima raiz de $p$.

In [ ]:
function trampa_newton_polinomios(Z, raices, eps = 0.05, TOPE = 1000)  
    cont = 0
    while cont <= TOPE
        for t in 1:length(raices) #para cada raiz revisar
            if abs(Z - raices[t]) <= eps #si esta cerca regresar un color HSV
                return HSV{Float64}((360.0*t)/length(raices), 1.0 ,exp(-0.05*cont))
                #el tono H depende de la raiz y el brillo V depende del tiempo
            end
        end
        S = sum([(1.0/(Z-R)) for R in raices])
        Z -= 1.0/S
        cont = cont + 1
    end
    return HSV{Float64}(0.0, 0.0, 0.0) #negro en cualquier otro caso
end

La función anterior, ``trampa_newton_polinomios``, toma como parametros el punto que se va a evaluar ``Z``, un arreglo de numeros complejos con las raices del polinomio ``raices``, el radio de las trampas ``eps`` y la cantidad máxima de itraciones permitidas ``TOPE``.

La función ``newton_polinomios`` aplica la trampa anterior a una muestra del plano complejo:

In [ ]:
function newton_polinomios(raices, eps = 0.05, TOPE = 1000, radio=2.0, centro=0.0im, alto=1000, ancho=1000)
 broadcast(Z -> trampa_newton_polinomios(Z, raices, eps, TOPE), muestra(radio, centro, alto, ancho))
end

Hagamos el dibujito del método de Newton para el polinomio $p(z) = z^N - 1$.
Para esto hacemos un arreglo ``mis_raices`` con la $N$-ésimas raices de la unidad y pasamos este arreglo a nuestra función ``newton_polinomios``.

In [ ]:
N = 5
mis_raices = [exp(2.0im*pi*i/N) for i = 1:N]
newton_polinomios(mis_raices, 0.1, 500, 3.0)

Ahora inicializamos el arreglo ``rand_raices`` con numeros aleatorios, con sus partes real e imaginaria distribuidas uniformemente en el intervalo $[-1, 1]$.

In [ ]:
N = 15
rand_raices = [rand(Uniform(-1,1)) + 1.0im*rand(Uniform(-1,1)) for i = 1:N]
newton_polinomios(rand_raices)

# Espacio de parametros del método de Newton para polinomios cúbicos

El método de Newton para polinomios cuadráticos esta conjugado a la función $z \mapsto z^2$ por un mapeo Möbius que lleva las raices a los puntos $0$ e $\infty$.

Consideremos polinomios cúbicos con al menos dos raíces distintas.
Salvo transformaciones afines del plano podemos suponer que los puntos $0$ y $1$ son raices del polinomio cúbico. El espacio de parámetros será la tercera raiz $\rho$, de esta forma el polinomio es $$p_{\rho}(z) = z(z - 1)(z- \rho).$$

El método de Newton para $p_{\rho}$ es una función racional de la forma $$N_{\rho}(z) = z - \frac{1}{\frac{1}{z} + \frac{1}{z-1} + \frac{1}{z-\rho}}.$$ 
Los puntos críticos de $N_{\rho}$ son las raices de $p_{\rho}$, $\{ 0, 1, \rho \}$, y el punto de inflexión (o baricentro) $\frac{1+\rho}{3}$. 

Conocemos la dinámica de las raíces; son puntos fijos superatractores. La dinamica del cuarto punto critico la podemos pintar en un dibujito. En este ejemplo la muestra será del espacio de parámetros $\rho$ y usaremos un algoritmo de escape (trampas) para perseguir la órbita del punto critico libre y, en caso de que converja a alguna raiz, pintaremos de un color dependiendo de a que raiz converge y cuanto tiempo le toma.

In [ ]:
function trampa_newton_param3(rho, eps=0.005, TOPE=1000)
    cont = 0
    Z = (rho + 1.0)/3.0 #el valor inicial es el punto critico libre
    while cont <= TOPE
        if abs(Z) < eps 
            #si se acerca a la raiz cero
            return HSV{Float64}(0.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - 1.0) < eps 
            #si se acerca a la raiz uno
            return HSV{Float64}(240.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - rho) < eps
            #si se acerca a la raiz rho
            return HSV{Float64}(120.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        end
        Z -= 1.0/((1.0/Z)+(1.0/(Z-1.0))+(1.0/(Z-rho))) #Newton
        cont += 1
    end
    return HSV{Float64}(0.0, 0.0 ,0.0) #negro en cualquier otro caso
end

In [ ]:
function newton_param3(eps = 0.005, TOPE = 1000, radio=0.75, centro=0.5+0.5im, alto=1000, ancho=1000)
    broadcast(Z -> trampa_newton_param3(Z, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
newton_param3()

Veamos unos zoom-zoom de la imagen anterior:

In [ ]:
newton_param3(0.05, 1500, 0.05, 0.816+0.58im)

In [ ]:
newton_param3(0.05, 2000, 0.014, 0.7895+0.6063im)

Veamos el dibujito de la dinamica del método de newton para el parametro del centro del dibujo anterior, cercano al 1/3-brazo del mandelbrotcito.

In [ ]:
newton_polinomios([0.0, 1.0, 0.7895+0.6063im], 0.05, 100, 0.55, 0.5+0.2im)

## ¿Qué pasa en el caso de grado 4?

Para empezar, el espacio de parámetros es de dimensión compleja 2, por lo que no podemos hacer un dibujito de todo el espacio de parámetros. Pero podemos expermientar.

Consideremos la subfamilia de polinomios de grado 4 dada por $q_{\rho} (z) = (z^3 - 1)(z - \rho)$. Los puntos criticos del método de Newton para esta familia, además de las raíces de $q_{\rho}$, son los puntos $0$ y $\frac{\rho}{2}$.

Ahora bien, el método de Newton toma la forma $$N_{q_{\rho}}(z) = \frac{3 z^4 - 2 \rho z^3 - \rho}{4 z^3- 3 \rho z^2 -1}$$ y tenemos que $$ N_{q_{\rho}} : 0 \mapsto \rho \mapsto \rho.$$

Veamos que pasa con el punto crítico $\frac{\rho}{2}$ haciendo el dibujito:

In [ ]:
function trampa_newton4_experimental(rho, eps=0.005, TOPE=1000)
    cont = 0
    Z = rho/2.0 #el valor inicial es el punto critico libre
    ω = cos(2.0*pi/3.0) + 1.0im*sin(2.0*pi/3.0) #la raiz primitiva cubica
    while cont <= TOPE
        if abs(Z - 1.0) < eps 
            #si se acerca a la raiz uno
            return HSV{Float64}(0.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - ω) < eps 
            #si se acerca a la raiz ω
            return HSV{Float64}(120.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - ω*ω) < eps
            #si se acerca a la raiz ω^2
            return HSV{Float64}(240.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - rho) < eps
            #si se acerca a la raiz rho
            return HSV{Float64}(60.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        end
        Z = (-rho + Z*Z*Z*(-2.0*rho + 3.0*Z))/(-1.0 + Z*Z*(-3.0*rho + 4.0*Z)) #Newton
        cont += 1
    end
    return HSV{Float64}(0.0, 0.0 ,0.0) #negro en cualquier otro caso
end

In [ ]:
function newton_param4_expermiental(eps = 0.005, TOPE = 1000, radio=5.0, centro=0.0, alto=1000, ancho=1000)
    broadcast(Z -> trampa_newton4_experimental(Z, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
newton_param4_expermiental()

De nuevo, veamos unos zoom-zoom:

In [ ]:
newton_param4_expermiental(0.005, 1500, 0.8, 1.4+1.9im)

In [ ]:
newton_param4_expermiental(0.005, 2000, 0.05, 1.457+1.902im)

Veamos el dibujito del método de Newton para $q_{\rho}$ con el parámetro del centro del dibujo anterior.

In [ ]:
ω = cos(2.0*pi/3.0) + 1.0im*sin(2.0*pi/3.0) #la raiz primitiva cubica
newton_polinomios([1.0, ω, ω*ω, 1.457+1.902im], 0.05, 200, 2.0, 0.4+0.4im)

## ¿Qué pasa con otras *rebanadas* del espacio de parámetros?

Ahora consideremos otra familia uni-parametrica de polinomios de grado cuatro. Sea ahora $q_{\rho} (z) = z(z^2 +1)(z - \rho)$.

El método de Newton para $q_{\rho}$ es $$ N_{q_{\rho}}(z) = \frac{3 z^4 - 2 \rho z^3 + z^2}{4 z^3 - 3 \rho z^2 + 2 z -\rho}. $$
Sus puntos críticos, ademas de las raíces, son: $$ \frac{3 \rho \pm \sqrt{9 \rho^2 - 24} }{12}$$ 

Hagamos el dibujito correspondiente al punto critico +:

In [ ]:
function raiz_cuadrada_newton(Z)
    t = angle(Z)
    if t < 0
        t = 2.0*pi + t
    end
    z0 = cos(t/2.0) + sin(t/2.0)*1.0im
    z1 = (z0/2.0) + Z/(2.0*z0)
    while abs(z0 - z1) > 1.0e-10
        z0 = z1
        z1 = (z0/2.0) + Z/(2.0*z0)
    end
    return z1
end

function trampa_newton4_experimental2(rho, eps=0.005, TOPE=1000)
    cont = 0
    #el valor inicial es el punto critico libre
    #Z = (3.0*rho + sqrt(9.0*rho*rho - 24.0))/12.0 
    Z =  (3.0*rho + raiz_cuadrada_newton(9.0*rho*rho - 24.0))/12.0 
    while cont <= TOPE
        if abs(Z) < eps 
            #si se acerca a la raiz cero
            return HSV{Float64}(0.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - 1.0im) < eps 
            #si se acerca a la raiz i
            return HSV{Float64}(120.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z + 1.0im) < eps
            #si se acerca a la raiz -i
            return HSV{Float64}(240.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        elseif abs(Z - rho) < eps
            #si se acerca a la raiz rho
            return HSV{Float64}(60.0, 1.0 ,exp(-(50.0/TOPE)*cont))
        end
        Z = (Z*Z*(1.0 + Z*(-2.0*rho + 3.0*Z)))/(-rho + Z*(2.0 + Z*(-3.0*rho + 4.0*Z))) #Newton
        cont += 1
    end
    return HSV{Float64}(0.0, 0.0 ,0.0) #negro en cualquier otro caso
end

In [ ]:
function newton_param4_expermiental2(eps = 0.005, TOPE = 1000, radio=2.0, centro=0.0, alto=1000, ancho=1000)
    broadcast(Z -> trampa_newton4_experimental2(Z, eps, TOPE), muestra(radio, centro, alto, ancho))
end

In [ ]:
newton_param4_expermiental2()

Modifique usted el codigo anterior para usar el otro punto critico.

¿Que pasa si usamos la funcion ``sqrt`` para calcular la raiz cuadrada en el computo del punto critico?

Veamos unos zoom-zoom del espacio de parametros:

In [ ]:
newton_param4_expermiental2(0.005, 1500, 0.025, 0.5 +0.25im)

Veamos el dibujito del método de Newton para el parámetro en el centro del dibujito anterior y tambień para el conjugado complejo del mismo parámetro.

In [ ]:
newton_polinomios([0.0, 1.0im, -1.0im, 0.5+0.25im], 0.05, 100, 2.0, 0.0)

In [ ]:
newton_polinomios([0.0, 1.0im, -1.0im, 0.5-0.25im], 0.05, 100, 2.0, 0.0)